<p align="center"><img width="50%" src="https://aimodelsharecontent.s3.amazonaws.com/aimodshare_banner.jpg" /></p>


---




<p align="center"><h1 align="center">Boston Housing Prices Regression Tutorial</h1> <h3 align="center">(Prepare to deploy model and preprocessor to REST API/Web Dashboard in four easy steps...)</h3></p>
<p align="center"><img width="80%" src="https://aimodelsharecontent.s3.amazonaws.com/ModelandPreprocessorObjectPreparation.jpeg" /></p>


---



## **(1) Preprocessor Function & Setup**




In [ ]:
! pip install scikit-learn --upgrade # Load newest version of sklearn.

     |████████████████████████████████| 6.8MB 2.5MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [ ]:
import pandas as pd
from sklearn import datasets, linear_model

# Obtaining the Boston Housing Prices dataset...
boston = datasets.load_boston()
X = pd.DataFrame(boston.data)

X.columns = boston.feature_names
y = boston.target # Or Price, i.e. median value of a house to be predicted.
X.head(2)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.9,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.9,9.14


In [ ]:
# Set up training and test data...
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 1987)

print(X_train.shape)
print(y_train.shape)
print(X.columns.tolist())

(404, 13)
(404,)
['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']


In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# We create the preprocessing pipelines for both numeric and categorical data.
numeric_features = ['CRIM', 'ZN', 'INDUS', 'NOX', 'RM', 'AGE', 'DIS',  'TAX', 'PTRATIO', 'B', 'LSTAT']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_features = ['CHAS', 'RAD']

# Replacing missing values with Modal value and then one hot encoding.
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Final preprocessor object set up with ColumnTransformer.
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])


# Fit your preprocessor object.
prediction_input_preprocessor=preprocessor.fit(X_train) 

import pickle
pickle.dump(prediction_input_preprocessor, open("preprocessor.pkl", "wb"))

### **Write a Preprocessor Function**

In [ ]:
# This is our preprocessor function to save using ai.export_preprocessor()...
def preprocessor(data):
  preprocessed_data=prediction_input_preprocessor.transform(data)
  return preprocessed_data

## **(2) Build Your Model**

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
model = Sequential()
model.add(Dense(32, input_dim=22, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16))
model.add(Dense(8))
model.add(Dense(4))
model.add(Dense(2))
model.add(Dense(1))

sgd = SGD(lr=0.001) # Learning rate for optimization set to 0.001.

# Compile model...
model.compile(loss='mse', optimizer=sgd, metrics=['mse'])

# Fitting the ANN to the Training set...
model.fit(preprocessor(X_train), y_train, 
               batch_size=32, 
               epochs=320, verbose=2)

Epoch 1/320
13/13 - 0s - loss: 263.9630 - mse: 263.9630
Epoch 2/320
13/13 - 0s - loss: 134.9353 - mse: 134.9353
Epoch 3/320
13/13 - 0s - loss: 44.4583 - mse: 44.4583
Epoch 4/320
13/13 - 0s - loss: 54.6474 - mse: 54.6474
Epoch 5/320
13/13 - 0s - loss: 36.1740 - mse: 36.1740
Epoch 6/320
13/13 - 0s - loss: 36.6099 - mse: 36.6099
Epoch 7/320
13/13 - 0s - loss: 26.5400 - mse: 26.5400
Epoch 8/320
13/13 - 0s - loss: 41.3476 - mse: 41.3476
Epoch 9/320
13/13 - 0s - loss: 26.6015 - mse: 26.6015
Epoch 10/320
13/13 - 0s - loss: 35.7710 - mse: 35.7710
Epoch 11/320
13/13 - 0s - loss: 21.2827 - mse: 21.2827
Epoch 12/320
13/13 - 0s - loss: 15.9027 - mse: 15.9027
Epoch 13/320
13/13 - 0s - loss: 16.6077 - mse: 16.6077
Epoch 14/320
13/13 - 0s - loss: 20.4599 - mse: 20.4599
Epoch 15/320
13/13 - 0s - loss: 26.8827 - mse: 26.8827
Epoch 16/320
13/13 - 0s - loss: 16.5627 - mse: 16.5627
Epoch 17/320
13/13 - 0s - loss: 11.6181 - mse: 11.6181
Epoch 18/320
13/13 - 0s - loss: 7.8944 - mse: 7.8944
Epoch 19/320
13/1

In [ ]:
model.predict(preprocessor(X_test))

array([[21.993082],
       [21.761253],
       [10.310246],
       [25.950556],
       [19.207293],
       [23.505026],
       [19.505232],
       [13.390075],
       [46.56502 ],
       [18.026236],
       [21.36947 ],
       [20.193367],
       [15.167277],
       [ 9.763607],
       [17.742172],
       [22.325336],
       [11.052958],
       [22.89468 ],
       [19.418995],
       [24.387182],
       [20.781782],
       [29.231243],
       [ 9.282041],
       [18.531254],
       [21.965158],
       [27.697096],
       [17.572123],
       [20.194084],
       [11.715147],
       [18.63393 ],
       [26.625349],
       [13.789065],
       [15.539425],
       [28.377356],
       [16.78662 ],
       [15.345576],
       [15.843083],
       [25.002707],
       [19.726068],
       [18.174925],
       [25.865255],
       [21.9546  ],
       [18.482027],
       [14.168027],
       [21.37304 ],
       [13.39111 ],
       [20.691542],
       [21.515924],
       [28.338057],
       [11.8245  ],


In [ ]:
y_pred = model.predict(preprocessor(X_test))

score = model.evaluate(preprocessor(X_test), y_test)
print('Accuracy: {:4f}'.format(score[1]))

4/4 [==============================] - 0s 2ms/step - loss: 10.0303 - mse: 10.0303
Accuracy: 10.030313


## **(3) Save Preprocessor**

In [ ]:
# ! pip3 install aimodelshare

In [ ]:
def export_preprocessor(preprocessor_function, filepath):
    import dill
    with open(filepath, "wb") as f:
        dill.dump(preprocessor_function, f)

# import aimodelshare as ai # Once we can deploy this, we use it in lieu of the below.
# ai.export_preprocessor(preprocessor, "preprocessor.pkl")

export_preprocessor(preprocessor, "preprocessor.pkl")

## **(4) Save Keras Model to Onnx File Format**

In [ ]:
! pip3 install keras2onnx
! pip3 install onnxruntime

     |████████████████████████████████| 102kB 2.6MB/s 
     |████████████████████████████████| 71kB 5.5MB/s 
     |████████████████████████████████| 7.4MB 9.0MB/s 
     |████████████████████████████████| 81kB 8.8MB/s 
  Created wheel for fire: filename=fire-0.3.1-py2.py3-none-any.whl size=111005 sha256=498d65e51db6254a7b8923d116826e5990587fcde8fd428e7f590dc27608de91
  Stored in directory: /root/.cache/pip/wheels/c1/61/df/768b03527bf006b546dce284eb4249b185669e65afc5fbb2ac
Successfully built fire
     |████████████████████████████████| 4.4MB 2.8MB/s 


In [ ]:
# Save model to onnx file...

import os
os.environ['TF_KERAS'] = '1' # Add this environmental variable whenever you use tensorflow's tf.keras to build your keras model.

import onnx
import keras2onnx

# Convert model to onnx object
import onnx
from keras2onnx import convert_keras
onnx_model = convert_keras(model, 'my_model.onnx')

# Save model to local .onnx file
with open("my_model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 25 -> 16
The maximum opset needed by this model is only 9.
